In [1]:
import numpy as np
from numpy.random import default_rng
import pandas as pd
from make_standings import Standings
from make_charts import make_playoff_charts, make_win_charts
from itertools import permutations
import time
import base64

In [6]:
df = pd.read_csv("schedules/schedule21.csv")
pr_both = pd.read_csv("data/pr_both.csv",squeeze=True,index_col="Side")
div_series = pd.read_csv("data/divisions.csv",squeeze=True,index_col=0)
teams = sorted(list(set(df.team_home)))
conf_teams = {}
for conf in ["AFC","NFC"]:
    conf_teams[conf] = [t for t in teams if div_series[t][:3]==conf]
rng = np.random.default_rng()

In [12]:
"ARI_Off" in pr_both.keys()

True

In [10]:
pr_both.keys()

Index(['ARI_Off', 'ATL_Off', 'BAL_Off', 'BUF_Off', 'CAR_Off', 'CHI_Off',
       'CIN_Off', 'CLE_Off', 'DAL_Off', 'DEN_Off', 'DET_Off', 'GB_Off',
       'HOU_Off', 'IND_Off', 'JAX_Off', 'KC_Off', 'LAC_Off', 'LAR_Off',
       'LVR_Off', 'MIA_Off', 'MIN_Off', 'NE_Off', 'NO_Off', 'NYG_Off',
       'NYJ_Off', 'PHI_Off', 'PIT_Off', 'SEA_Off', 'SF_Off', 'TB_Off',
       'TEN_Off', 'WAS_Off', 'ARI_Def', 'ATL_Def', 'BAL_Def', 'BUF_Def',
       'CAR_Def', 'CHI_Def', 'CIN_Def', 'CLE_Def', 'DAL_Def', 'DEN_Def',
       'DET_Def', 'GB_Def', 'HOU_Def', 'IND_Def', 'JAX_Def', 'KC_Def',
       'LAC_Def', 'LAR_Def', 'LVR_Def', 'MIA_Def', 'MIN_Def', 'NE_Def',
       'NO_Def', 'NYG_Def', 'NYJ_Def', 'PHI_Def', 'PIT_Def', 'SEA_Def',
       'SF_Def', 'TB_Def', 'TEN_Def', 'WAS_Def', 'HFA', 'mean_score'],
      dtype='object', name='Side')

In [8]:
div_series

ARI     NFC West
ATL    NFC South
BAL    AFC North
BUF     AFC East
CAR    NFC South
CHI    NFC North
CIN    AFC North
CLE    AFC North
DAL     NFC East
DEN     AFC West
DET    NFC North
GB     NFC North
HOU    AFC South
IND    AFC South
JAX    AFC South
KC      AFC West
LAC     AFC West
LAR     NFC West
LVR     AFC West
MIA     AFC East
MIN    NFC North
NE      AFC East
NO     NFC South
NYG     NFC East
NYJ     AFC East
PHI     NFC East
PIT    AFC North
SEA     NFC West
SF      NFC West
TB     NFC South
TEN    AFC South
WAS     NFC East
Name: 0, dtype: object

In [7]:
def simulate_reg_season(df):
    df.loc[:,["score_home","score_away"]] = rng.normal(df[["mean_home","mean_away"]],10)
    df.loc[:,["score_home","score_away"]] = df.loc[:,["score_home","score_away"]].astype(int)
    adjust_ties(df)
    return None

def adjust_ties(df):
    tied_games = df.loc[df.score_home == df.score_away,["score_home","score_away"]].copy()
    x = rng.normal(size=len(tied_games))
    tied_games.iloc[np.where((x > -1.6) & (x < 0))[0]] += (0,3)
    tied_games.iloc[np.where(x >= 0)[0]] += (3,0)
    df.loc[tied_games.index,["score_home","score_away"]] = tied_games
    return None

In [ ]:
df["pr_home_Off"] = df.team_home.map(lambda s: pr_both[s+"_Off"])
df["pr_home_Def"] = df.team_home.map(lambda s: pr_both[s+"_Def"])
df["pr_away_Off"] = df.team_away.map(lambda s: pr_both[s+"_Off"])
df["pr_away_Def"] = df.team_away.map(lambda s: pr_both[s+"_Def"])
df["mean_home"] = df["pr_home_Off"]-df["pr_away_Def"]+pr_both["mean_score"]+pr_both["HFA"]/2
df["mean_away"] = df["pr_away_Off"]-df["pr_home_Def"]+pr_both["mean_score"]-pr_both["HFA"]/2

In [ ]:
total_dict = {}

win_dict = {t:{i:0 for i in range(18)} for t in teams}
for conf in ["AFC","NFC"]:
    total_dict[conf] = {i:{t:0 for t in sorted(conf_teams[conf])} for i in range(1,8)}

In [ ]:
div_dict = {}

s = div_series
for a,b in s.groupby(s):
    div_dict[a] = list(b.index)

rank_dict = {div:{} for div in div_dict.keys()}

for div in rank_dict.keys():
    for team_sort in permutations(div_dict[div]):
        rank_dict[div][team_sort] = 0

In [ ]:
reps = 30

start = time.time()

for i in range(reps):
    simulate_reg_season(df)
    stand = Standings(df)
    s = stand.standings
    s_dict = dict(zip(s.index,s.Wins))
    for t,w in s_dict.items():
        win_dict[t][w] += 1
    for d in rank_dict.keys():
        rank_dict[d][tuple(stand.div_ranks[d])] += 1
    p = stand.playoffs
    for conf in ["AFC","NFC"]:
        for j,t in enumerate(p[conf]):
            total_dict[conf][j+1][t] += 1

end = time.time()

In [ ]:
stand.div_ranks

In [ ]:

            
for div in stand.div_ranks.keys():
            rank_dict[div][tuple(stand.div_ranks[div])] += 1

In [ ]:
rank_dict

In [ ]:
def prob_to_odds(p):
    if p < .000001:
        return "NA"
    if p > 0.5:
        x = 100*p/(p-1)
        return f"{x:.0f}"
    elif p <= 0.5:
        x = 100*(1-p)/p
        return f"+{x:.0f}"

In [ ]:
prob_to_odds(0)

In [ ]:
color_dict = {"blue": "#5778a4", "orange": "#e49444", "red": "#d1615d", "teal": "#85b6b2", "green": "#6a9f58", "yellow": "#e7ca60", "purple": "#a87c9f",}

chart_dict = {}

for conf in ["AFC","NFC"]:

    playoff_dicts = total_dict[conf]

    source = pd.DataFrame([(k,t,playoff_dicts[k][t]/reps) for k in playoff_dicts.keys() for t in conf_teams[conf]],
        columns = ["Seed","Team","Proportion"])

    for a,b in source.groupby("Team"):
        source.loc[source["Team"] == a, "Make playoffs"] = b.Proportion.sum()
        source.loc[source["Team"] == a,"Equal or better"] = b.Proportion.cumsum()

    #source["Equal or better"] += source["Proportion"]

    ordering = sorted(conf_teams[conf],key=lambda t: sum([playoff_dicts[i][t] for i in playoff_dicts.keys()]),reverse=True)
    ordering_seeds = list(range(7,0,-1))


    c = alt.Chart(source).mark_bar().encode(
        x=alt.X('Team',sort = ordering),
        y=alt.Y('Proportion',scale=alt.Scale(domain=[0,1]),sort=ordering_seeds),
        tooltip = [alt.Tooltip('Team'),alt.Tooltip('Seed', format=".0f"), alt.Tooltip('Proportion', format=".3f"),
                            alt.Tooltip('Equal or better', format=".3f"),alt.Tooltip('Make playoffs', format=".3f")],
        color=alt.Color('Seed:N', scale=alt.Scale(domain=[7,6,5,4,3,2,1],range=[color_dict[color] for color in ["purple","yellow","green","teal","red","orange","blue"]])),
        order=alt.Order(
                'Seed:N',
                sort='ascending'
            )
    ).properties(
        title=f"{conf} playoff seedings",
        width=500,
        height=500,
    )

    chart_dict[conf] = c

playoff_charts = alt.hconcat(*chart_dict.values()).resolve_scale(
    color='independent'
).properties(
    title=f"Based on {reps} simulations:"
)

In [ ]:
odds_dict = {"Proportion":"Odds", "Make playoffs": "Odds to make playoffs", "Equal or better":"Odds of equal or better"}

In [ ]:
for c in odds_dict.keys():
    source[odds_dict[c]] = source[c].map(prob_to_odds)

In [ ]:
alt.Chart(source).mark_point().encode(
    x = "Proportion",
    y = "Make playoffs",
    tooltip = [alt.Tooltip("x:N",title="chris"),"y:N"],
).transform_calculate(
    x="format(datum.Proportion, ',.3f')+' (' +datum.Odds+')'",
)

In [ ]:
source["Odds"]

In [ ]:
playoff_charts

In [ ]:
game_list = ["Mette", "Damir", "Amra", "Marie", "Chris"]

In [ ]:
import numpy as np
rng = np.random.default_rng()
rng.permutation(game_list)

In [ ]:
series = df.apply(lambda row: rng.permutation(game_list), axis=1)

In [ ]:
df

In [ ]:
def highlight_cols(s):
    color = 'yellow'
    return 'background-color: %s' % color

In [ ]:
df = pd.DataFrame(index=range(10**2),columns = range(5))
for i in range(len(df)):
    df.loc[i] = rng.permutation(game_list)
df = df.style.applymap(highlight_cols, subset=pd.IndexSlice[:, 0:2])

In [ ]:
df

In [ ]:
df.data = df.data.iloc[2:10]

In [ ]:
df